In [1]:
from Sqlite3Helpers import Sqlite3Functions as s3
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from MyanmarNLPTools import MMSegmentor, MMCleaner, MMTools
from tqdm import tqdm
import json

seg = MMSegmentor.MMSegmentor()
cln = MMCleaner.MMCleaner()

In [2]:
# Connect to the Sqlite3 database file
conn = s3.connect_to_sqlite3(filepath='../../Scraping/db/ghs.db')

In [3]:
df = s3.fetch(con=conn, query='''
SELECT * FROM etl1
''') # error between 6740 and 6741 for `etl`
print(df.shape)
display(df.head())

(1378355, 16)


,post_url,channel_url,datetime,edit_datetime,message,num_forwards,from_scheduled,fwd_from,mentioned,pinned,post,replies_replies,replies_replies_pts,reply_to,views,via_bot_id
0,https://t.me/justicseeker/7574,https://t.me/justicseeker,2022-08-02 18:57:41+00:00,2022-08-02 19:40:01+00:00,အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...,5.0,0.0,None,0,0.0,1,2.0,20355.0,None,1249.0,NaN
1,https://t.me/justicseeker/7573,https://t.me/justicseeker,2022-08-02 18:40:06+00:00,2022-08-02 18:46:18+00:00,,5.0,0.0,None,0,0.0,1,0.0,20355.0,None,1138.0,NaN
2,https://t.me/justicseeker/7572,https://t.me/justicseeker,2022-08-02 18:39:46+00:00,2022-08-02 23:27:56+00:00,[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။\nရပ်ပြစ်ရှစ်ပါး...,8.0,0.0,None,0,0.0,1,0.0,20355.0,None,1112.0,NaN
3,https://t.me/justicseeker/7571,https://t.me/justicseeker,2022-08-02 18:39:46+00:00,2022-08-02 20:46:44+00:00,🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏\nဗုဒ္ဓဘာသာ မှန်လျှင် သိရှ...,8.0,0.0,None,0,0.0,1,0.0,20355.0,None,933.0,NaN
4,https://t.me/justicseeker/7570,https://t.me/justicseeker,2022-08-02 18:30:41+00:00,2022-08-03 02:30:46+00:00,Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...,1.0,0.0,None,0,0.0,1,0.0,20355.0,None,915.0,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378355 entries, 0 to 1378354
Data columns (total 16 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   post_url             1378355 non-null  object 
 1   channel_url          1378355 non-null  object 
 2   datetime             1378355 non-null  object 
 3   edit_datetime        735526 non-null   object 
 4   message              1360241 non-null  object 
 5   num_forwards         1085316 non-null  float64
 6   from_scheduled       1360241 non-null  float64
 7   fwd_from             381318 non-null   object 
 8   mentioned            1378355 non-null  int64  
 9   pinned               1360241 non-null  float64
 10  post                 1378355 non-null  int64  
 11  replies_replies      560107 non-null   float64
 12  replies_replies_pts  560107 non-null   float64
 13  reply_to             230667 non-null   object 
 14  views                1085316 non-null  float64
 15

In [5]:
df.describe()

,num_forwards,from_scheduled,mentioned,pinned,post,replies_replies,replies_replies_pts,views,via_bot_id
count,1.085316e+06,1360241.0,1.378355e+06,1.360241e+06,1.378355e+06,560107.000000,560107.000000,1.085316e+06,4.180000e+02
mean,2.173965e+00,0.0,7.255025e-07,3.367639e-02,6.805467e-01,1.634173,247641.669085,2.599263e+03,2.033193e+08
std,9.570926e+00,0.0,8.517643e-04,1.803949e-01,4.662650e-01,14.696156,200443.421571,7.136122e+03,1.071902e+08
min,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,9.529350e+07
25%,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,55301.000000,3.300000e+02,1.906010e+08
50%,1.000000e+00,0.0,0.000000e+00,0.000000e+00,1.000000e+00,0.000000,166810.000000,1.038000e+03,1.906010e+08
75%,2.000000e+00,0.0,0.000000e+00,0.000000e+00,1.000000e+00,1.000000,480004.000000,2.438000e+03,1.906010e+08
max,3.049000e+03,0.0,1.000000e+00,1.000000e+00,1.000000e+00,3339.000000,567943.000000,2.965161e+06,2.137971e+09


In [6]:
df.via_bot_id.unique()

array([           nan, 1.90601014e+08, 2.13797100e+09, 8.96515845e+08,
       3.28179432e+08, 1.40267078e+08, 9.52935000e+07])

In [7]:
# Set datatypes
df['datetime_posted'] = pd.to_datetime(df.datetime).dt.tz_convert('Asia/Rangoon')
df['datetime_edited'] = pd.to_datetime(df.edit_datetime).dt.tz_convert('Asia/Rangoon')
df['message'] = df.message.astype(str)
df['num_forwards'] = df.num_forwards.fillna(0).astype(int)
df['from_scheduled'] = df.from_scheduled.astype(bool)
df['fwd_from'] = df.fwd_from.astype(str)
df['mentioned'] = df.mentioned.astype(int)
df['pinned'] = df.pinned.astype(bool)
df['post'] = df.post.astype(bool)
df['replies_replies'] = df.replies_replies.astype(float) # because of nan
df['replies_replies_pts'] = df.replies_replies_pts.astype(float) # because of nan
# df['reply_to'] = df.reply_to.astype(?)
df['views'] = df.views.fillna(0).astype(int)
# df['via_bot_id'] = df.via_bot_id.astype(?)
# df['date_posted'] = df.datetime.dt.date
# df['time_posted'] = df.datetime.dt.time
# df['date_edited'] = df.edit_datetime.dt.date
# df['time_edited'] = df.edit_datetime.dt.time

df.head()

,post_url,channel_url,datetime,edit_datetime,message,num_forwards,from_scheduled,fwd_from,mentioned,pinned,post,replies_replies,replies_replies_pts,reply_to,views,via_bot_id,datetime_posted,datetime_edited
0,https://t.me/justicseeker/7574,https://t.me/justicseeker,2022-08-02 18:57:41+00:00,2022-08-02 19:40:01+00:00,အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...,5,False,None,0,False,True,2.0,20355.0,None,1249,NaN,2022-08-03 01:27:41+06:30,2022-08-03 02:10:01+06:30
1,https://t.me/justicseeker/7573,https://t.me/justicseeker,2022-08-02 18:40:06+00:00,2022-08-02 18:46:18+00:00,,5,False,None,0,False,True,0.0,20355.0,None,1138,NaN,2022-08-03 01:10:06+06:30,2022-08-03 01:16:18+06:30
2,https://t.me/justicseeker/7572,https://t.me/justicseeker,2022-08-02 18:39:46+00:00,2022-08-02 23:27:56+00:00,[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။\nရပ်ပြစ်ရှစ်ပါး...,8,False,None,0,False,True,0.0,20355.0,None,1112,NaN,2022-08-03 01:09:46+06:30,2022-08-03 05:57:56+06:30
3,https://t.me/justicseeker/7571,https://t.me/justicseeker,2022-08-02 18:39:46+00:00,2022-08-02 20:46:44+00:00,🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏\nဗုဒ္ဓဘာသာ မှန်လျှင် သိရှ...,8,False,None,0,False,True,0.0,20355.0,None,933,NaN,2022-08-03 01:09:46+06:30,2022-08-03 03:16:44+06:30
4,https://t.me/justicseeker/7570,https://t.me/justicseeker,2022-08-02 18:30:41+00:00,2022-08-03 02:30:46+00:00,Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...,1,False,None,0,False,True,0.0,20355.0,None,915,NaN,2022-08-03 01:00:41+06:30,2022-08-03 09:00:46+06:30


In [8]:
data = df[['post_url', 'channel_url', 'datetime_posted','message','datetime_edited','views']]
data.rename(mapper={
    'message': 'msg_og'
}, axis=1, inplace=True)
data.head()

,post_url,channel_url,datetime_posted,msg_og,datetime_edited,views
0,https://t.me/justicseeker/7574,https://t.me/justicseeker,2022-08-03 01:27:41+06:30,အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...,2022-08-03 02:10:01+06:30,1249
1,https://t.me/justicseeker/7573,https://t.me/justicseeker,2022-08-03 01:10:06+06:30,,2022-08-03 01:16:18+06:30,1138
2,https://t.me/justicseeker/7572,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။\nရပ်ပြစ်ရှစ်ပါး...,2022-08-03 05:57:56+06:30,1112
3,https://t.me/justicseeker/7571,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏\nဗုဒ္ဓဘာသာ မှန်လျှင် သိရှ...,2022-08-03 03:16:44+06:30,933
4,https://t.me/justicseeker/7570,https://t.me/justicseeker,2022-08-03 01:00:41+06:30,Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...,2022-08-03 09:00:46+06:30,915


# Extract Outlinks

In [9]:
data['outlinks'] = data.msg_og.apply(MMTools.extract_urls)
data.head()

,post_url,channel_url,datetime_posted,msg_og,datetime_edited,views,outlinks
0,https://t.me/justicseeker/7574,https://t.me/justicseeker,2022-08-03 01:27:41+06:30,အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...,2022-08-03 02:10:01+06:30,1249,[]
1,https://t.me/justicseeker/7573,https://t.me/justicseeker,2022-08-03 01:10:06+06:30,,2022-08-03 01:16:18+06:30,1138,[]
2,https://t.me/justicseeker/7572,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။\nရပ်ပြစ်ရှစ်ပါး...,2022-08-03 05:57:56+06:30,1112,[]
3,https://t.me/justicseeker/7571,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏\nဗုဒ္ဓဘာသာ မှန်လျှင် သိရှ...,2022-08-03 03:16:44+06:30,933,[]
4,https://t.me/justicseeker/7570,https://t.me/justicseeker,2022-08-03 01:00:41+06:30,Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...,2022-08-03 09:00:46+06:30,915,[]


# Clean Messages

In [10]:
data['msg_clean'] = data.msg_og.apply(MMTools.remove_urls).apply(cln.web_clean) # `cln.web_clean` adds <newline> token for each \n
data.head()

,post_url,channel_url,datetime_posted,msg_og,datetime_edited,views,outlinks,msg_clean
0,https://t.me/justicseeker/7574,https://t.me/justicseeker,2022-08-03 01:27:41+06:30,အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...,2022-08-03 02:10:01+06:30,1249,[],အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...
1,https://t.me/justicseeker/7573,https://t.me/justicseeker,2022-08-03 01:10:06+06:30,,2022-08-03 01:16:18+06:30,1138,[],
2,https://t.me/justicseeker/7572,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။\nရပ်ပြစ်ရှစ်ပါး...,2022-08-03 05:57:56+06:30,1112,[],[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။<newline>ရပ်ပြစ်...
3,https://t.me/justicseeker/7571,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏\nဗုဒ္ဓဘာသာ မှန်လျှင် သိရှ...,2022-08-03 03:16:44+06:30,933,[],🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏<newline>ဗုဒ္ဓဘာသာ မှန်လျှ...
4,https://t.me/justicseeker/7570,https://t.me/justicseeker,2022-08-03 01:00:41+06:30,Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...,2022-08-03 09:00:46+06:30,915,[],Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...


# Normalize MM Text

In [11]:
zg_detected = 0
total_sentences = 0
msg_uni = []

for m in tqdm(data.msg_clean):
    row_lvl = []
    for l in m.split('<newline>'):
        total_sentences += 1
        if MMTools.is_zawgyi_probability(l) > 0.95:
            zg_detected += 1
            row_lvl.append(MMTools.zg2uni(l))
        else:
            row_lvl.append(l)
    msg_uni.append('<newline>'.join(row_lvl))

print('Written in ZG:', zg_detected, 'out of', total_sentences)
data['msg_clean'] = msg_uni

data.head()

  0%|          | 0/1378355 [00:00<?, ?it/s]/home/htetaung04/miniconda3/envs/mw/lib/python3.7/site-packages/myanmartools/zawgyi_detector.py:50: RuntimeWarning: overflow encountered in exp
  np.exp(self.params[previous[mask], current[mask]].sum()))
100%|██████████| 1378355/1378355 [02:14<00:00, 10262.18it/s]


Written in ZG: 182859 out of 7308860


,post_url,channel_url,datetime_posted,msg_og,datetime_edited,views,outlinks,msg_clean
0,https://t.me/justicseeker/7574,https://t.me/justicseeker,2022-08-03 01:27:41+06:30,အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...,2022-08-03 02:10:01+06:30,1249,[],အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...
1,https://t.me/justicseeker/7573,https://t.me/justicseeker,2022-08-03 01:10:06+06:30,,2022-08-03 01:16:18+06:30,1138,[],
2,https://t.me/justicseeker/7572,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။\nရပ်ပြစ်ရှစ်ပါး...,2022-08-03 05:57:56+06:30,1112,[],[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။<newline>ရပ်ပြစ်...
3,https://t.me/justicseeker/7571,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏\nဗုဒ္ဓဘာသာ မှန်လျှင် သိရှ...,2022-08-03 03:16:44+06:30,933,[],🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏<newline>ဗုဒ္ဓဘာသာ မှန်လျှ...
4,https://t.me/justicseeker/7570,https://t.me/justicseeker,2022-08-03 01:00:41+06:30,Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...,2022-08-03 09:00:46+06:30,915,[],Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...


# Clean Again after Unicode Conversion

In [12]:
data['msg_clean'] = data.msg_clean.apply(cln.web_clean).str.replace('_','').str.lower()
data.head()

,post_url,channel_url,datetime_posted,msg_og,datetime_edited,views,outlinks,msg_clean
0,https://t.me/justicseeker/7574,https://t.me/justicseeker,2022-08-03 01:27:41+06:30,အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...,2022-08-03 02:10:01+06:30,1249,[],အကြိမ် ၁ သန်းမှာ တစ်ကြိမ်ပဲ ရနိုင်တဲ့ ဓာတ်ပုံရ...
1,https://t.me/justicseeker/7573,https://t.me/justicseeker,2022-08-03 01:10:06+06:30,,2022-08-03 01:16:18+06:30,1138,[],
2,https://t.me/justicseeker/7572,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။\nရပ်ပြစ်ရှစ်ပါး...,2022-08-03 05:57:56+06:30,1112,[],[ အပြစ် ]ဟု မိန့်တော်မူသည် … ။<newline>ရပ်ပြစ်...
3,https://t.me/justicseeker/7571,https://t.me/justicseeker,2022-08-03 01:09:46+06:30,🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏\nဗုဒ္ဓဘာသာ မှန်လျှင် သိရှ...,2022-08-03 03:16:44+06:30,933,[],🔴 ဩကာသ ရှင်းတမ်း 🙏🙏🙏<newline>ဗုဒ္ဓဘာသာ မှန်လျှ...
4,https://t.me/justicseeker/7570,https://t.me/justicseeker,2022-08-03 01:00:41+06:30,Reactive Respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...,2022-08-03 09:00:46+06:30,915,[],reactive respond သည် သဘာဝဘေးအန္တရာယ်ကျရောက်ပြီ...


# Finding Dups

In [13]:
msg_dups = data.groupby('msg_clean').size().sort_values(ascending=False).reset_index()
msg_dups.columns = ['msg_clean', 'qty']
msg_dups = msg_dups.loc[msg_dups.qty > 1]
msg_dups = msg_dups.loc[msg_dups.msg_clean.str.len()>0]
msg_dups = msg_dups.loc[[m!='-' for m in msg_dups.msg_clean]]
print('Number of messages with dups:', msg_dups.shape[0])
msg_dups

Number of messages with dups: 107576


,msg_clean,qty
1,none,18114
2,this channel can’t be displayed because it vio...,13664
3,👍,11527
4,#whatshappeninginmyanmar,4011
5,t.me/channelasiabot,946
...,...,...
107573,>> honda fit အဖြူ ၁စီး ဗမရ မှ သဖပွိုင့်ဘက်သို့...,2
107574,sedona hotel ရှေ့မှာ ခွေး ၆ ကောင်ရှိ<newline>7...,2
107575,cc.7654proboxအဖြူအမှတ်၂စခန်းသာကေတကရဲခွေး၅ကောင်...,2
107576,pdf-kalay စခန်းသိမ်းပိုက်စဉ် အသေမိတဲ့ မ ၁ ဦးနဲ...,2


# Drop Dups through Post URLs

In [14]:
data.drop_duplicates(subset=['post_url'], inplace=True)
tmp = data.groupby('post_url').size().sort_values(ascending=False)
assert (tmp != 1).sum() == 0, 'There are duplicates.'

# Prepare Datatypes

In [15]:
data['outlinks'] = [json.dumps(l, ensure_ascii=False) for l in data.outlinks]

# Syllable Segmentation Chunkwise

__Issue__: The number of rows is too high that kernel crashes due to memory limit.

- Before syllable segmentation, save the data to drive.
- Delete the existing `etl_clean` table from Sqlite3 database since the chunk approach below will require to append instead replace.
- Read the saved file from drive in chunks.
- Each chunk get syllable-segmented and inserted into the Sqlite3 database.
- Adjust the chunk size until it's optimal.

In [16]:
# Save data to tmp file
filename = '../data/tmp/tmp.csv'
data.to_csv(filename, index=False)
# # Delete `data`
# try: del data
# except: pass

# Delete `etl_clean`
s3.q(con=conn, query='''
DROP TABLE IF EXISTS etl_clean;
''')

# Read csv in chunks
chunksize = 10000
for chunk in tqdm(pd.read_csv(filename, chunksize=chunksize)):
    chunk['msg_seg'] = [
        json.dumps([a for a in l if a])
        for l in chunk.msg_clean.fillna('').apply(seg.syllable_segment)]
    s3.insert(table='etl_clean', con=conn, df=chunk, if_exists='append', chunksize=chunksize)

134it [03:40,  1.64s/it]


: 